# 126 Project Traffic Simulator

Let the road be length $n$. Cars start at position 0 and reach their destination at position $n-1$. Each car has an exponential "clock" with parameter $\gamma$, and every time their clock goes off they move forward with probability $p$. If a car is directly in front of them, they do not move regardless. A car enters the road at position 0 at a rate of $Exp(\alpha)$ and exit the road at position $n-1$ with a rate of $Exp(\beta)$.

### Choose global variables:

In [1]:
from ipywidgets import interactive
from IPython.display import display
from IPython.display import clear_output

def set_params(n, alpha, beta, theta, gamma, p, seed):
    return [n, alpha, beta, theta, gamma, p, seed]

params = interactive(set_params, n=(1, 20), alpha=(0.0, 2.0), beta=(0.0, 2.0), theta=(0.0, 0.5), gamma=(0.0, 2.0), p=(0.0, 1.0), seed=(1, 999))
display(params)

interactive(children=(IntSlider(value=10, description='n', max=20, min=1), FloatSlider(value=1.0, description=…

### Set global variables:
(Note: will break if you set alpha = 0 due to 0 division error)
Remember to rerun this cell every time a slider is updated

In [2]:
n, alpha, beta, theta, gamma, p, seed = params.result

### Define particle and environment classes:

In [7]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output


class Particle:
    
    count = 0
    
    def __init__(self, time, gamma=gamma, p=p):
        Particle.count += 1
        self.id = Particle.count - 1
        self.gamma = gamma
        self.p = p
        self._position = 0
        
        self._time = time # time at which the particle went to this position
        
    def update_position(self, time, value=1):
        self._position += value
        
        self._time = time # update time
        
    def get_position(self):
        return self._position
        
class Environment:
    def __init__(self, n=n, alpha=alpha, beta=beta):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.particles = []
        self.occupied_positions = [0 for _ in range(n)]
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 # slight issue where running repeated simulations wouldn't cause particle counts to reset
        
        sys.stdout.write("\r{}".format(self.occupied_positions))
        sys.stdout.flush()
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/self.alpha)
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle(curr_time, gamma=gamma, p=p)
        self.particles.append(particle)
        self.occupied_positions[0] = 1
        
        actions = [(curr_time, particle.id, particle.get_position())]
        entrances = 1
        exits = 0
        
        travel_times = [[0, None]]
        cell_speeds = []
        
        while curr_time < time_len:
            sys.stdout.write("\r{}".format(self.occupied_positions))
            sys.stdout.flush()
            
            # should we include alpha/beta in the merged_param if the first spot is occupied/the last spot is empty?
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.alpha + self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha, self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                if self.occupied_positions[0]:
                    continue
                else:
                    new_particle = Particle(curr_time, gamma=gamma, p=p)
                    self.particles.append(new_particle)
                    self.occupied_positions[0] = 1
                    
                    actions += [(curr_time, new_particle.id, new_particle.get_position())]
                    entrances += 1
                    
                    travel_times.append([curr_time, None])
                    
            elif particle == 'exit':
                if self.occupied_positions[self.n-1]: # changed to refer to self.n instead of n
                    end_particle = self.particles.pop(0)
                    self.occupied_positions[self.n-1] = 0 # changed to refer to self.n instead of n
                    
                    actions += [(curr_time, end_particle.id, 'exit')]
                    exits += 1
                    
                    travel_times[end_particle.id][1] = curr_time
                    
                else:
                    continue
            else:
                particle_pos = particle.get_position()
                if particle_pos == self.n - 1 or self.occupied_positions[particle_pos + 1] == 1: # changed to refer to self.n instead of n
                    continue
                else:
                    self.occupied_positions[particle_pos] = 0
                    self.occupied_positions[particle_pos + 1] = 1
                    
                    cell_speeds.append(1 / (curr_time - particle._time))
                    particle.update_position(curr_time)
                    
                    actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times = [end - begin for begin, end in travel_times if end]
        if len(travel_times) > 0:
            print("Travel times: {}".format([round(t, 2) for t in travel_times]))
            print('Average travel time: {}'.format(sum(travel_times) / len(travel_times)))
        else:
            print('No cars reached the end')
        if len(cell_speeds) > 0:
            print('Average cell speed: {}'.format(sum(cell_speeds) / len(cell_speeds)))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Add 2D Environment:

In [9]:
class Particle2D(Particle):
    def __init__(self, gamma=gamma, p=p, direction=0): # 0 is horizontal, 1 is vertical
        Particle.__init__(self, gamma, p)
        self._position = [-1, -1]
        self._direction = direction
        
    def update_position(self, value=1):
        self._position[self._direction] += value
        
    def set_position(self, value=[-1, -1]):
        self._position = value
        
    def get_direction(self):
        return self._direction
        
# FIRST INDEX COLUMN, SECOND INDEX ROAD
class Env2D(Environment):
    def __init__(self, num_roads=(1,1), n=n, alpha=alpha, beta=beta, theta=theta):
        Environment.__init__(self, n, alpha, beta)
        self.theta = theta
        self.occupied_positions = np.array([[0 for _ in range(n)] for _ in range(n)])
        self.horizontal_roads = np.random.choice(range(n), num_roads[0], replace=False)
        self.vertical_roads = np.random.choice(range(n), num_roads[1], replace=False)
        self.num_roads = num_roads
        self.intersections = []
        [[self.intersections.append((x, y)) for x in self.vertical_roads] for y in self.horizontal_roads]
        self.inter_directions = {}
        for i in self.intersections:
            self.inter_directions[i] = 0
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 # slight issue where running repeated simulations wouldn't cause particle counts to reset
        
        print(self.occupied_positions)
        
        curr_time = 0
        time_lapse = np.random.exponential(scale=1/(self.alpha * sum(self.num_roads)))
        time.sleep(time_lapse)
        curr_time += time_lapse
        particle = Particle2D(gamma=gamma, p=p, direction=np.random.choice([0, 1], p=[self.num_roads[0]/sum(self.num_roads), self.num_roads[1]/sum(self.num_roads)]))
        self.particles.append(particle)
        if not particle.get_direction():
            road = np.random.choice(self.horizontal_roads)
            particle.set_position([0, road])
            self.occupied_positions[0, road] = 1
        else:
            road = np.random.choice(self.vertical_roads)
            particle.set_position([road, 0])
            self.occupied_positions[road, 0] = 1
        
        actions = [(curr_time, particle.id, particle.get_position())]
        entrances = 1
        exits = 0
        
        while curr_time < time_len:
            clear_output(wait=True)
            print(self.occupied_positions)
            
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + (self.alpha + self.beta) * sum(self.num_roads) + self.theta * (sum(self.num_roads) ** 2)  
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] + [self.alpha for _ in range(sum(self.num_roads))] + [self.beta for _ in range(sum(self.num_roads))] + [self.theta for _ in range(sum(self.num_roads) ** 2)]) / merged_param
            particle = np.random.choice(self.particles + ['entrance' for _ in range(sum(self.num_roads))] + ['exit' for _ in range(sum(self.num_roads))] + ['intersection' for _ in range(sum(self.num_roads) ** 2)], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[0][road]:
                        continue
                    new_particle = Particle2D(gamma=gamma, p=p, direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[0][road] = 1
                    new_particle.set_position([0, road])
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][0]:
                        continue
                    new_particle = Particle2D(gamma=gamma, p=p, direction=direction)
                    self.particles.append(new_particle)
                    self.occupied_positions[road][0] = 1
                    new_particle.set_position([road, 0])
                
                    
                actions += [(curr_time, new_particle.id, new_particle.get_position())]
                entrances += 1
                    
            elif particle == 'exit':
                direction = np.random.choice([0, 1], p=[len(self.horizontal_roads)/sum(self.num_roads), len(self.vertical_roads)/sum(self.num_roads)])
                if not direction:
                    road = np.random.choice(self.horizontal_roads)
                    if self.occupied_positions[self.n-1][road]: # changed to refer to self.n instead of n
                        particle = [p for p in self.particles if p.get_position() == [self.n-1, road]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[self.n-1][road] = 0 # changed to refer to self.n instead of n
                        actions += [(curr_time, particle.id, [n, road])]
                        exits += 1
                else:
                    road = np.random.choice(self.vertical_roads)
                    if self.occupied_positions[road][self.n-1]: # changed to refer to self.n instead of n
                        particle = [p for p in self.particles if p.get_position() == [road, self.n-1]][0]
                        self.particles.remove(particle)
                        self.occupied_positions[road][self.n-1] = 0 # changed to refer to self.n instead of n
                        actions += [(curr_time, particle.id, [road, n])]
                        exits += 1
            elif particle == 'intersection':
                i = np.random.choice(range(len(self.intersections)))
                intersection = self.intersections[i]
                self.inter_directions[intersection] = 0 if self.inter_directions[intersection] else 1
                actions += [(curr_time, 'intersection' + str(intersection), self.inter_directions[intersection])] 
            else:
                particle_pos = particle.get_position()
                direction = particle.get_direction()
                if not direction:
                    pos = (particle_pos[0] + 1, particle_pos[1])
                    if pos in self.intersections and self.inter_directions[pos]:
                        continue
                    elif particle_pos[0] == self.n - 1 or self.occupied_positions[particle_pos[0] + 1, particle_pos[1]] == 1:
                        continue
                    else:
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0] + 1][particle_pos[1]] = 1
                        particle.update_position()
                        actions += [(curr_time, particle.id, particle.get_position())]
                else:
                    pos = (particle_pos[0], particle_pos[1] + 1)
                    if pos in self.intersections and not self.inter_directions[pos]:
                        continue
                    elif particle_pos[1] == self.n - 1 or self.occupied_positions[particle_pos[0], particle_pos[1] + 1] == 1:
                        continue
                    else:
                        self.occupied_positions[particle_pos[0]][particle_pos[1]] = 0
                        self.occupied_positions[particle_pos[0]][particle_pos[1] + 1] = 1
                        particle.update_position()
                        actions += [(curr_time, particle.id, particle.get_position())]
                    
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(entry[0]))
            print('Particle: {}'.format(entry[1]))
            print('New position: {}'.format(entry[2]))
            print('\n')

### Multiple Lanes Environment:

In [103]:
class ParticleManyLane(Particle):
    def __init__(self, time, lane, gamma=gamma, p=p):
        Particle.__init__(self, time, gamma, p)
        self._position = [lane, 0] #first index lane, second index position
        
    def update_position(self, time, value=(0, 1)):
        self._time = time
        self._position[0] += value[0]
        self._position[1] += value[1]
        
    def set_position(self, value=[-1, -1]):
        self._position = value
        
    def get_position(self):
        return self._position[0], self._position[1]
    
class EnvManyLane(Environment):
    def __init__(self, n=n, alpha=alpha, beta=beta, lanes=1, delta=0.5):
        Environment.__init__(self, n, alpha, beta)
        self.lanes = lanes
        self.occupied_positions = np.zeros((lanes, n))
        self.delta = delta # tendency to change lanes
        #first index lane, second index position
        
    def run_simulation(self, time_len=60):
        Particle.count = 0 
        
        curr_time = 0
        entrances = 0
        exits = 0
        
        actions = []
        travel_times = [[0, None]]
        cell_speeds = []
        
        while curr_time < time_len:
            clear_output(wait=True)
            print(self.occupied_positions)
            
            # should we include alpha/beta in the merged_param if the first spot is occupied/the last spot is empty?
            merged_param = sum([particle.gamma * particle.p for particle in self.particles]) + self.lanes * self.alpha + self.lanes * self.beta
            time_lapse = np.random.exponential(scale=1/merged_param)
            probabilities = np.array([particle.gamma * particle.p for particle in self.particles] +  [self.lanes * self.alpha, self.lanes * self.beta]) / merged_param
            particle = np.random.choice(self.particles + ['entrance', 'exit'], p=probabilities)
            time.sleep(time_lapse)
            curr_time += time_lapse
            if particle == 'entrance':
                
                lane = np.random.randint(0, self.lanes)
                if self.occupied_positions[lane, 0]:
                    continue
                else:
                    new_particle = ParticleManyLane(curr_time, lane, gamma=gamma, p=p)
                    self.particles.append(new_particle)
                    self.occupied_positions[lane, 0] = 1
                    
                    actions += [(curr_time, new_particle.id, new_particle.get_position())]
                    entrances += 1
                    
                    travel_times.append([curr_time, None])
                    
            elif particle == 'exit':
                
                lane = np.random.randint(0, self.lanes)
                if self.occupied_positions[lane, self.n-1]: # changed to refer to self.n instead of n
                    for i in range(len(self.particles)):
                        particle = self.particles[i]
                        if particle.get_position() == (lane, self.n-1):
                            end_particle = particle
                            self.particles.pop(i)
                            break
                    self.occupied_positions[lane, self.n-1] = 0 # changed to refer to self.n instead of n
                    
                    actions += [(curr_time, end_particle.id, 'exit')]
                    exits += 1
                    
                    travel_times[end_particle.id][1] = curr_time
                    
                else:
                    continue
            else:
                particle_lane, particle_pos = particle.get_position()
                options = []
                if particle_pos != self.n - 1 and self.occupied_positions[particle_lane, particle_pos + 1] != 1:
                    options.append((0, 1)) # advance
                if particle_lane - 1 >= 0 and self.occupied_positions[particle_lane - 1, particle_pos - 1] == 0 and self.occupied_positions[particle_lane - 1, particle_pos] == 0:
                    options.append((-1, 0)) # left
                if particle_lane + 1 < self.lanes and self.occupied_positions[particle_lane + 1, particle_pos - 1] == 0 and self.occupied_positions[particle_lane + 1, particle_pos] == 0:
                    options.append((1, 0)) # right
                    
                if len(options) == 0:
                    continue
                else:
                    probabilities = []
                    if (0, 1) in options:
                        probabilities.append((self.n - np.sum(self.occupied_positions[particle_lane, :])) * (1 - self.delta))
                    if (-1, 0) in options:
                        probabilities.append((self.n - np.sum(self.occupied_positions[particle_lane - 1, :])) * self.delta)
                    if (1, 0) in options:
                        probabilities.append((self.n - np.sum(self.occupied_positions[particle_lane + 1, :])) * self.delta)
                    print(probabilities)
                    probabilities = probabilities / sum(probabilities)
                    print(probabilities)
                    option = options[np.random.choice(len(options), p=probabilities)]
                    self.occupied_positions[particle_lane, particle_pos] = 0
                    self.occupied_positions[particle_lane + option[0], particle_pos + option[1]] = 1
                    if option == (0, 1):
                        cell_speeds.append(1 / (curr_time - particle._time))
                        particle.update_position(curr_time, (0, 1))
                    else:
                        particle.update_position(particle._time, (option))
                    actions += [(curr_time, particle.id, particle.get_position())]
                          
        print('\n done')
        print('Summary:')
        print('Total entrances: {}'.format(entrances))
        print('Total exits: {}'.format(exits))
        travel_times = [end - begin for begin, end in travel_times if end]
        if len(travel_times) > 0:
            print("Travel times: {}".format([round(t, 2) for t in travel_times]))
            print('Average travel time: {}'.format(sum(travel_times) / len(travel_times)))
        else:
            print('No cars reached the end')
        if len(cell_speeds) > 0:
            print('Average cell speed: {}'.format(sum(cell_speeds) / len(cell_speeds)))
        print('\n Log:')
        for entry in actions:
            print('Time: {}'.format(round(entry[0], 2)), end = " ")
            print('Particle: {}'.format(entry[1]), end = " ")
            print('New lane, position: ({},{})'.format(entry[2][0], entry[2][1]), end = " ")
            print('\n')

### Run simulator:

In [101]:
env = Env2D(num_roads=(3,3), n=n, alpha=alpha, beta=beta, theta=theta)
env.run_simulation(time_len = 10)

[[1 1 0 1 0 0 0 0 1 0]
 [0 1 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

 done
Summary:
Total entrances: 12
Total exits: 0

 Log:
Time: 0.06560150948295305
Particle: 0
New position: [3, 1]


Time: 0.10110651402530355
Particle: intersection(5, 3)
New position: 1


Time: 0.21853613781938935
Particle: intersection(7, 3)
New position: 1


Time: 0.22934168346513747
Particle: intersection(0, 1)
New position: 1


Time: 0.376763893283556
Particle: 0
New position: [3, 1]


Time: 0.3842766135977757
Particle: intersection(5, 8)
New position: 1


Time: 0.3991411224423381
Particle: 1
New position: [1, 3]


Time: 0.4229549040798289
Particle: 2
New position: [0, 0]


Time: 0.5638416289874549
Particle: intersection(7, 3)
New position: 0


Time: 0.8656029962100867
Particle: 3
New position: [5, 2]


Time: 1.0469767784770356
Particle: 4
New positio

In [55]:
env1 = Environment(n=n, alpha=alpha, beta=beta)
env1.run_simulation(time_len = 1)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 done
Summary:
Total entrances: 1
Total exits: 0
No cars reached the end

 Log:
Time: 0.4365311643593912
Particle: 0
New position: 0




In [104]:
env2 = EnvManyLane(n=n, alpha=alpha, beta=beta, lanes=2)
env2.run_simulation(time_len = 40)

[[0. 1. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1. 0.]]
[2.5]
[1.]

 done
Summary:
Total entrances: 14
Total exits: 1
Travel times: [16.02]
Average travel time: 16.02377696799747
Average cell speed: 1.3782542173272476

 Log:
Time: 1.03 Particle: 0 New lane, position: (0,0) 

Time: 1.44 Particle: 1 New lane, position: (1,0) 

Time: 2.34 Particle: 0 New lane, position: (0,1) 

Time: 2.51 Particle: 1 New lane, position: (1,1) 

Time: 2.65 Particle: 1 New lane, position: (1,2) 

Time: 2.71 Particle: 2 New lane, position: (1,0) 

Time: 2.77 Particle: 1 New lane, position: (1,3) 

Time: 2.84 Particle: 3 New lane, position: (0,0) 

Time: 4.06 Particle: 2 New lane, position: (1,1) 

Time: 4.26 Particle: 4 New lane, position: (1,0) 

Time: 5.78 Particle: 0 New lane, position: (0,2) 

Time: 6.11 Particle: 0 New lane, position: (0,3) 

Time: 6.61 Particle: 0 New lane, position: (0,4) 

Time: 6.62 Particle: 2 New lane, position: (1,2) 

Time: 6.72 Particle: 2 New lane, position: (0,2) 
